In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [5]:
def get_max_sizes(*data_sets):
    max_sent_size = max_ques_size = max_fact_count = 0
    for data in data_sets:
        for x, q, fc in zip(data.xs, data.qs, data.fact_counts):
            for fact in x: max_sent_size = max(max_sent_size, len(fact))
            max_ques_size = max(max_ques_size, len(q))
            max_fact_count = max(max_fact_count, fc)

    return max_sent_size, max_ques_size, max_fact_count

In [8]:
import os

In [ ]:
def get_args():
    parser = argparse.ArgumentParser()
    home = os.path.expanduser("~")
    source_dir = os.path.join(home, "data", "squad")
    target_dir = "home/Rishab/Desktop//data/squad"
    glove_dir = os.path.join(home, "data", "glove")
    parser.add_argument('-s', "--source_dir", default=source_dir)
    parser.add_argument('-t', "--target_dir", default=target_dir)
    parser.add_argument('-d', "--debug", action='store_true')
    parser.add_argument("--train_ratio", default=0.9, type=int)
    parser.add_argument("--glove_corpus", default="6B")
    parser.add_argument("--glove_dir", default=glove_dir)
    parser.add_argument("--glove_vec_size", default=100, type=int)
    parser.add_argument("--mode", default="full", type=str)
    parser.add_argument("--single_path", default="", type=str)
    parser.add_argument("--tokenizer", default="PTB", type=str)
    parser.add_argument("--url", default="vision-server2.corp.ai2", type=str)
    parser.add_argument("--port", default=8000, type=int)
    parser.add_argument("--split", action='store_true')
    # TODO : put more args here
    return parser.parse_args()

In [37]:
f=open("/home/Rishab/Desktop/squad/data/sequence/train.txt",'r')

In [39]:
x=f.readline()
x

"Architecturally , the school has a Catholic character . Atop the Main Building 's gold dome is a golden statue of the Virgin Mary . Immediately in front of the Main Building and facing it , is a copper statue of Christ with arms upraised with the legend `` Venite Ad Me Omnes '' . Next to the Main Building is the Basilica of the Sacred Heart . Immediately behind the basilica is the Grotto , a Marian place of prayer and reflection . It is a replica of the grotto at Lourdes , France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858 . At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ) , is a simple , modern stone statue of Mary .\tWhat is in front of the Notre Dame Main Building ?\t38 39 40 41 42\n"

In [47]:
x.split("\t")[2]

'38 39 40 41 42\n'

In [40]:
import nltk
cv=nltk.word_tokenize(x)
cnt=-1
for i in cv:
    cnt+=1
    print(i,cnt)

In [32]:
def prepSQuAD():
    reload(sys)
    sys.setdefaultencoding('utf-8')
    import json
    from nltk.tokenize import word_tokenize
    count = 0
    filenames = ['dev', 'train']
    for filename in filenames:
        fpr = open("/home/Rishab/Desktop/squad/data/"+filename+"-v1.1.json", 'r')
        line = fpr.readline()
        js = json.loads(line)
        fpw = open("/home/Rishab/Desktop/squad/data/sequence/"+filename+".txt", 'wr')
        for c in js["data"]:
            for p in c["paragraphs"]:
                context = p["context"].split(' ')
                context_char = list(p["context"])
                context_pos = {}
                for qa in p["qas"]:
                    question = word_tokenize(qa["question"])
                    if filename == 'train':
                        for a in qa['answers']:
                            answer = a['text'].strip()
                            answer_start = int(a['answer_start'])
                            answer_words = word_tokenize(answer+'.')
                            if answer_words[-1] == '.':
                                answer_words = answer_words[:-1]
                            else:
                                answer_words = word_tokenize(answer)
                        prev_context_words = word_tokenize( p["context"][0:answer_start ] )
                        left_context_words = word_tokenize( p["context"][answer_start:] )
                        answer_reproduce = []
                        for i in range(len(answer_words)):
                            if i < len(left_context_words):
                                w = left_context_words[i]
                                answer_reproduce.append(w)
                        join_a = ' '.join(answer_words)
                        join_ar = ' '.join(answer_reproduce)
                        if join_a != join_ar:
                            count += 1
                        fpw.write(' '.join(prev_context_words+left_context_words)+'\t')
                        fpw.write(' '.join(question)+'\t')
                        
                        pos_list = []
                        for i in range(len(answer_words)):
                            if i < len(left_context_words):
                                pos_list.append(str(len(prev_context_words)+i+1))
                        if len(pos_list) == 0:
                            print join_ar
                            print join_a
                            print 'answer:'+answer
                        assert(len(pos_list) > 0)
                        fpw.write(' '.join(pos_list)+'\n')
                    else:
                        fpw.write(' '.join(word_tokenize( p["context"]) )+'\t')
                        fpw.write(' '.join(question)+'\n')
                

        fpw.close()
    print ('SQuAD preprossing finished!')

In [ ]:



opt.model = 'boundaryMPtr'
opt.task = 'squad'
opt.log = 'log information'
opt.visualize = false
opt.numWords = 124164
torch.manualSeed(opt.seed)



In [ ]:
tf.app.flags.DEFINE_integer("batch_size",6,"")
tf.app.flags.DEFINE_integer("wvecdim",300,"")
tf.app.flags.DEFINE_integer("mem_dim",150,"")
tf.app.flags.DEFINE_integer("att_dim",150,"")
tf.app.flags.DEFINE_integer("num_processes",5,"")
tf.app.flags.DEFINE_integer("max_epoches",30,"")
tf.app.flags.DEFINE_integer("seed",123,"")
tf.app.flags.DEFINE_integer("reg",0,"")
tf.app.flags.DEFINE_boolean("visualize",False,"")
tf.app.flags.DEFINE_integer("numwords",124164,"")
tf.app.flags.DEFINE_integer("num_layers",1,"")
tf.app.flags.DEFINE_integer("m_layers",2,"")
tf.app.flags.DEFINE_float("learning_rate",0.002,"")
tf.app.flags.DEFINE_float("lr_decay",0.002,"")
tf.app.flags.DEFINE_float("dropoutP",0.4,"")
tf.app.flags.DEFINE_integer("emb_lr",0,"")
tf.app.flags.DEFINE_string("preEmb","glove","")
tf.app.flags.DEFINE_string("grad","adamax","")
tf.app.flags.DEFINE_integer("expIdx",0,"")
tf.app.flags.DEFINE_boolean("inititlaUNK",False,"")
tf.app.flags.DEFINE_boolean("emb_partial",True,"")

FLAGS=tf.app.flags.FLAGS


In [12]:
import sys

In [ ]:
with tf.variable_scope('Question'):
            ques_list = tf.unpack(tf.transpose(question))
            ques_embed = [tf.nn.embedding_lookup(embedding, w) for w in ques_list]
            _, question_vec = tf.nn.rnn(gru, ques_embed, dtype=tf.float32)

In [5]:
from tensorflow.python.ops import rnn_cell

In [103]:
import tensorflow.python.ops

In [3]:
from tensorflow.python.ops

In [2]:
def _get_dims(shape):
    fan_in = shape[0] if len(shape) == 2 else np.prod(shape[:-1])
    fan_out = shape[1] if len(shape) == 2 else shape[-1]
    return fan_in, fan_out

def weight(name, shape, init='he', range=None):
    
    initializer = tf.constant_initializer()
    if init == 'xavier':
        fan_in, fan_out = _get_dims(shape)
        range = math.sqrt(6.0 / (fan_in + fan_out))
        initializer = tf.random_uniform_initializer(-range, range)

    elif init == 'he':
        fan_in, _ = _get_dims(shape)
        std = math.sqrt(2.0 / fan_in)
        initializer = tf.random_normal_initializer(stddev=std)

    
    var = tf.get_variable(name, shape, initializer=initializer)
    tf.add_to_collection('l2', tf.nn.l2_loss(var)) 
    return var


In [3]:
def bias(name, dim, initial_value=0.0):
    dims = dim if isinstance(dim, list) else [dim]
    return tf.get_variable(name, dims, initializer=tf.constant_initializer(initial_value))

In [95]:
with tf.variable_scope("apple"):
    cell=rnn_cell.LSTMCell(100,state_is_tuple=True)
    H_paara = tf.placeholder(tf.float32, shape=[40, 60 , 80 ], name='para') 
    axx,_=tf.nn.dynamic_rnn(cell,H_paara,dtype=tf.float32)


In [160]:
seqlen=tf.placeholder(tf.int32,[64],"ghyu")
with tf.variable_scope("amipoom"):
    H_par4a = tf.placeholder(tf.float32, shape=[64, 20 , 80 ], name='para') 
    embedding = weight('di789aoosd', [100, 80], init='uniform', range=3**(1/2))
    cell=rnn_cell.LSTMCell(80,state_is_tuple=True)
    para_list=tf.unpack(tf.transpose(H_para))
    para_embed=[tf.nn.embedding_lookup(embedding,w) for w in para_list]
    xPara , _ =tf.nn.dynamic_rnn(cell,H_par4a,seqlen,dtype=tf.float32)

In [161]:
xPara

<tf.Tensor 'amipoom/RNN/transpose:0' shape=(64, 20, 80) dtype=float32>

In [74]:
class DMN(BaseModel):
    
    
    def build(self):
        
        N="batch size"; P="para length"; Q="ques length"; aa="answ length"
        V="words embedding size"
        A="words vocab length"
        d="hidden size"
        
        H_para = tf.placeholder('int32', shape=[N, P , V ], name='para') 
        H_ques = tf.placeholder('int32', shape=[N, Q , V], name='ques')  
        H_answ = tf.placeholder('int32', shape=[N, aa], name='answ')
        
        
        para_mask = tf.placeholder('float32', shape=[N, P, V], name='para_m')
        ques_mask = tf.placeholder('float32', shape=[N, Q, V], name='ques_m')
        
                
        # Prepare parameters
        cell = rnn_cell.LSTMCell(d,state_is_tuple=True)
        #l = self.positional_encoding()
        embedding = weight('embedding', [A, V], init='uniform', range=3**(1/2))
        
        

                
        with tf.variable_scope('Para'):
            para_list=tf.unpack(tf.transpose(H_para))
            para_embed=[tf.nn.embedding_lookup(embedding,w) for w in para_list]
            
            Para , _ =tf.nn.dynamic_rnn(cell,H_para,dtype=tf.float32) # [batch_size, para_length , hidden_state(d)]
            #Para, para_state = tf.nn.rnn(cell, para_embed, dtype=tf.float32)  # [ batch_size, hidden_state ]
            
        with tf.variable_scope('ques'):
            ques_list=tf.unpack(tf.transpose(H_ques))
            ques_embed=[tf.nn.embedding_lookup(embedding,w) for w in ques_list]
            Ques , _ =tf.nn.dynamic_rnn(cell,H_ques,dtype=tf.float32) # [batch_size, ques_length , hidden_state(d)]
            
            Ques, Ques_state = tf.nn.rnn(cell, ques_embed, dtype=tf.float32) # [ batch_size, hidden_state ]
        

        
        MatchLstm()
        Answerpointer()
        

        with tf.name_scope('Loss'):
            cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, answer)
            loss = tf.reduce_mean(cross_entropy)
            total_loss = loss + params.weight_decay * tf.add_n(tf.get_collection('l2'))
            
            
            
        
        
        

SyntaxError: invalid syntax (<ipython-input-74-cc6fbb45a706>, line 42)

In [215]:
#HR = weight('hr', [d, 1])
class MatchLstm():
    def __init__(self, num_hidden, question, facts, is_training, bn):
        self.WQ = weight('wq', [num_hidden, num_hidden])
        self.WP = weight('wp', [num_hidden, num_hidden])
        self.WR = weight('wr', [num_hidden, num_hidden])
        self.HR = weight('hr', [num_hidden, 1])
        self.b =  weight('b' ,  [1])
        self.bp = bias('bp', [num_hidden, 1])
        self.w  = bias('w',  [num_hidden, 1])
        self.b = bias('b', 1)
        self.facts_transposed = [tf.transpose(f) for f in self.facts]  
        
    def init_state(self):
        return tf.zeros_like(self.facts_transposed[0])
    
    def cal_hidd():
        t_cell=rnn_cell.BasicLSTMCell(self.num_hidden)
        with tf.variable_scope(''):
            #calculation of G
            WQXHQ = tf.matmul(self.WQ, Ques)
            WPXHP  =  tf.matmul(self.WP, h_para)
            WPXHP +=  tf.matmul(self.WR, hr)
            WPXHP +=  self.bp
            l1= tf.tile(WPXHP , [1, Q] )  #[ hidden_size , question_length ]
            l_g=tf.nn.tanh(l1)
            
            #calculation of alpha
            
            wt = tf.transpose(self.w)       #[1 , hidden_size     ]
            l2 = tf.matmul(wt,l_g)          #[1 , question_length ]
            leq = tf.tile(self.b , [1, Q])  #[1 , question_length ]
            l_alpha = tf.nn.softmax(l2 + leq)  #[1 , question_length ]
            
            
            #calculation of z
            
            t_alpha=tf.transpose(l_alpha)
            temp_var=tf.matmul(self.ques,t_alpha)  #[hidden_state , 1]
            #temp_z=tf.concat(0,[h_para,temp_var]) # to be discussed later
            with tf.variable_scope('final_hidden') as final_h_scope:
                cell = rnn_cell.LSTMCell(d,state_is_tuple=True)
                temp_z=tf.transpose(temp_var)
                _, hidden_state = tf.nn.rnn(cell, [temp_z], dtype=tf.float32)
                final_h_scope.reuse_variables()

In [ ]:
class Answer_pointer():
    
    def __init__(hidden_size,H_r):
        self.V  =  weight('V', [ hidden_size, 2*hidden_size ])
        self.WA =  weight('WA',[ hidden_size, hidden_size ])
        self.ba =  weight('ba',[ hidden_size, 1 ])
        self.v  =  weight('v',[ hidden_size, 1 ])
        self.facts_transposed = [tf.transpose(f) for f in self.facts]
        self.ze_ten = tf.zeros([hidden_size,1])
        self.H_r = H_r
        self.H_rr = tf.concat(1, [self.H_r,self.ze_ten] )
        self.c =    weight('c',[ 1 ])
        
        
    
    def init_state(self):
        return tf.zeros_like(self.facts_transposed[0])
    
    def cal_ans(H_r):
        
        # calcualtion of FK
        l1   = tf.matmul(self.WA , self.HA)
        l1+  = self.ba
        lep1 = tf.tile( l1 , [1 , (P+1) ])
        
        l2  = tf.matmul(self.V, self.H_rr)
        lf2 = l2 + lep1
        
        #calculation of  beta
        temp_vfk = tf.matmul(tf.transpose(self.v), lf2 )
        temp_cfk = tf.tile(self.c , [1,(P+1) ])
        beta  = tf.nn.softmax(temp_vfk + temp_cfk)
        
        #calculation of Hak
        
        beta_t = tf.transpose(beta)
        temp_h = tf.matmul(self.H_rr,beta_t )  # [ 2* hidden_size , 1]
        
        with tf.variable_scope(""):
            cell = rnn_cell.LSTMCell(d,state_is_tuple=True)
            temp_z=tf.transpose(temp_var)
            _, hidden_state = tf.nn.rnn(cell, [temp_z], dtype=tf.float32)
            final_h_scope.reuse_variables()
            
        
        
        
        
        
        

In [218]:
rft= tf.placeholder('int32',[40,8])
rt = tf.placeholder('int32',[40,1])

In [162]:
def get_state_variables(batch_size, cell):
    state_variables = []
    for state_c, state_h in cell.zero_state(batch_size, tf.float32):
        state_variables.append(tf.nn.rnn_cell.LSTMStateTuple(
            tf.Variable(state_c, trainable=False),
            tf.Variable(state_h, trainable=False)))
    return tuple(state_variables)


def get_state_update_op(state_variables, new_states):
    update_ops = []
    for state_variable, new_state in zip(state_variables, new_states):
        update_ops.extend([state_variable[0].assign(new_state[0]),
                           state_variable[1].assign(new_state[1])])
    return tf.tuple(update_ops)

In [ ]:
data = tf.placeholder(tf.float32, (batch_size, max_length, frame_size))
cell_layer = tf.nn.rnn_cell.GRUCell(256)
cell = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers)

states = get_state_variables(batch_size, cell)
outputs, new_states = tf.nn.dynamic_rnn(cell, data, initial_state=states)
update_op = get_state_update_op(states, new_states)

sess = tf.Session()

In [64]:
ap

[<tf.Tensor 'RNN_2/BasicLSTMCell/mul_2:0' shape=(8, 4) dtype=float32>]

In [55]:
hat=tf.placeholder(tf.int16,[3,2])
tah=tf.placeholder(tf.int16,[3,2])

In [56]:
tf.concat(0,[hat,tah])

<tf.Tensor 'concat_28:0' shape=(6, 2) dtype=int16>

In [32]:
ad = bias('ad', 80 )


<function __main__.bias>

In [41]:
axd = weight('axd', [80, 1])

In [43]:
axd.get_shape()

TensorShape([Dimension(80), Dimension(1)])

In [37]:
import numpy as np
import pandas as pd


In [34]:
tf.placeholder("int32", [80,1],'as')

<tf.Tensor 'as_1:0' shape=(80, 1) dtype=int32>

In [1]:
import tensorflow as tf

In [5]:
from tensorflow.python.ops import rnn_cell

In [6]:
H_para = tf.placeholder('int32', shape=[64, 20, 80 ], name='para') 

In [8]:
cell = rnn_cell.BasicLSTMCell(60)
embedding = weight('embedding', [800, 80], init='uniform', range=3**(1/2))

In [10]:
used = tf.sign(tf.reduce_max(tf.abs(H_para), reduction_indices=2))
length = tf.cast(tf.reduce_sum(used, reduction_indices=1), tf.int32)

In [16]:
used,length

(<tf.Tensor 'Sign:0' shape=(64, 20) dtype=int32>,
 <tf.Tensor 'Sum:0' shape=(64,) dtype=int32>)

In [18]:
cell.output_size

60

In [26]:
output = tf.reshape(tf.transpose(tf.pack(output), perm=[1, 0, 2]), [-1,  60])

In [27]:
output

<tf.Tensor 'Reshape:0' shape=(1280, 120) dtype=float32>

In [25]:
output.s

[<tf.Tensor 'concat:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_1:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_2:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_3:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_4:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_5:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_6:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_7:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_8:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_9:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_10:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_11:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_12:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_13:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_14:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_15:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_16:0' shape=(64, 120) dtype=float32>,
 <tf.Tensor 'concat_17:0' 

In [13]:
H_para

<tf.Tensor 'para:0' shape=(64, 20, 80) dtype=int32>

In [20]:
length

<tf.Tensor 'Sum:0' shape=(64,) dtype=int32>

In [7]:
import numpy as np

In [8]:
vrc=np.zeros([64,10])

In [ ]:
preprocessing()
match_lstm()
answer_ponter()
    
    

In [85]:
#self.params.embed_size, self.params.max_sent_size, self.params.batch_size
def positional_encoding():
        D=80
        M=20  
        N=64
        encoding = np.zeros([M, D])
        for j in range(M):
            for d in range(D):
                encoding[j, d] = (1 - float(j)/M) - (float(d)/D)*(1 - 2.0*j/M)

        return encoding

In [100]:
l.shape

(20, 80)

In [95]:
#N F L 

N=64            #batch size
F=1            #fact_count
L=20           #max sent size
d=80            #hidden dimension
V=80            #word embedding
    

In [96]:
H_para = tf.placeholder('int32', shape=[N, F , L ], name='para') 
input_mask = tf.placeholder('float32', shape=[N, F, L, V], name='xm')
embedding = weight('em64', [100, V], init='uniform', range=3**(1/2))

with tf.name_scope('SentenceReader'):
    input_list = tf.unpack(tf.transpose(H_para))  
    input_embed = []
    for facts in input_list:
        facts = tf.unpack(facts)
        embed = tf.pack([tf.nn.embedding_lookup(embedding, w) for w in facts]) 
        input_embed.append(embed)

        
l=positional_encoding()

input_embed = tf.transpose(tf.pack(input_embed), [2, 1, 0, 3])  # [N, F, L, V]
encoded = l * input_embed * input_mask
facts = tf.reduce_sum(encoded, 2)  # [N, F, V]

            
                                       

In [97]:
gru = rnn_cell.GRUCell(d)
fact_cout = tf.placeholder('int64', shape=[N], name='fc')
with tf.name_scope('InputFu'):
    with tf.variable_scope('Forw'):
        forward_tas, _ = tf.nn.dynamic_rnn(gru, facts, fact_cout, dtype=tf.float32)

In [98]:
forward_tas

<tf.Tensor 'InputFu/Forw/RNN/transpose:0' shape=(64, 20, 80) dtype=float32>

In [82]:
facts

<tf.Tensor 'Sum:0' shape=(64, 12, 60) dtype=float32>

In [66]:
Para

[<tf.Tensor 'ert/RNN/BasicLSTMCell/mul_2:0' shape=(64, 80) dtype=float32>,
 <tf.Tensor 'ert/RNN/BasicLSTMCell_1/mul_2:0' shape=(64, 80) dtype=float32>,
 <tf.Tensor 'ert/RNN/BasicLSTMCell_2/mul_2:0' shape=(64, 80) dtype=float32>,
 <tf.Tensor 'ert/RNN/BasicLSTMCell_3/mul_2:0' shape=(64, 80) dtype=float32>,
 <tf.Tensor 'ert/RNN/BasicLSTMCell_4/mul_2:0' shape=(64, 80) dtype=float32>,
 <tf.Tensor 'ert/RNN/BasicLSTMCell_5/mul_2:0' shape=(64, 80) dtype=float32>,
 <tf.Tensor 'ert/RNN/BasicLSTMCell_6/mul_2:0' shape=(64, 80) dtype=float32>,
 <tf.Tensor 'ert/RNN/BasicLSTMCell_7/mul_2:0' shape=(64, 80) dtype=float32>,
 <tf.Tensor 'ert/RNN/BasicLSTMCell_8/mul_2:0' shape=(64, 80) dtype=float32>,
 <tf.Tensor 'ert/RNN/BasicLSTMCell_9/mul_2:0' shape=(64, 80) dtype=float32>,
 <tf.Tensor 'ert/RNN/BasicLSTMCell_10/mul_2:0' shape=(64, 80) dtype=float32>,
 <tf.Tensor 'ert/RNN/BasicLSTMCell_11/mul_2:0' shape=(64, 80) dtype=float32>]